<a href="https://colab.research.google.com/github/andrevcmelo/BootcampAlura2021/blob/main/BootcampAlura_ProjetoModulo03_Andr%C3%A9Melo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Projeto do Módulo 03 do Bootcamp Alura em Data Science Aplicada 2020**

**Aluno: [André Victor Costa de Melo](https://https://www.linkedin.com/in/andrevcmelo/)**

O objetivo deste projeto é realizar uma análise exploratória na base de dados da pesquisa PeNSE de 2015, do IBGE, utilizando conhecimentos de estatística e visualização de dados.

1. Informações sobre a PeNSE 2015 - https://www.ibge.gov.br/estatisticas/sociais/populacao/9134-pesquisa-nacional-de-saude-do-escolar.html?=&t=o-que-e

2. IBGE Explica - PeNSE: https://www.youtube.com/watch?v=z4BBCHZUvD4

3. Microdados Download: https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html?caminho=pense/2015/microdados/

4. Link Dicionário dos Dados: https://github.com/andrevcmelo/BootcampAlura2021/blob/main/bases_modulo3/Dicionario_PENSE_Microdados_AMOSTRA2.xls

5. Link Base de Dados: https://raw.githubusercontent.com/andrevcmelo/BootcampAlura2021/main/bases_modulo3/PENSE_AMOSTRA2_ALUNOESCOLA.CSV

# **Resumo**

A Pesquisa Nacional de Saúde do Escolar - PeNSE teve início em 2009, fruto de parceria com o Ministério da Saúde e apoio do Ministério da Educação. Na edição de 2015, que utilizaremos como base de dados para este projeto, se destaca a disponibilização de informações oriundas de dois planos amostrais distintos: 1) escolares frequentando o 9o ano do ensino fundamental e **2)** escolares de 13 a 17 anos de idade frequentando as etapas do 6o ao 9o ano do ensino fundamental (antigas 5a a 8a séries) e da 1a a 3a série do ensino médio. Neste projeto iremos utilizar a **amostra 2**.

A pesquisa fornece informações sobre as características básicas da população de estudo, divididas entre:
1. `Dados dos alunos:` incluindo aspectos socioeconômicos, como escolaridade dos pais, inserção no mercado de trabalho e posse de bens e serviços; contextos social e familiar; fatores de risco comportamentais relacionados a hábitos alimentares, sedentarismo, tabagismo, consumo de álcool e outras drogas; saúde sexual e reprodutiva; exposição a acidentes e violências; hábitos de higiene; saúde bucal; saúde mental; e percepção da imagem corporal, entre outros tópicos. 
2. `Dados das escolas:` Características do ambiente escolar e do entorno são também contempladas, incluindo informações relacionadas à infraestrutura disponível para alimentação e atividade física; acessibilidade; saneamento básico; existência de regras e normas de conduta adotadas pelas escolas; políticas de assistência à saúde; e nível de segurança do entorno, entre outros aspectos.

Diante deste cenário da base de dados, irei avaliar algumas hipóteses através de análises exploratórias nos dados. As hipóteses que guiarão este projeto são:

1. Os adolescentes brasileiros estão satisfeitos com seus corpos?
2. Há uma alta frequência de bullying nas escolas?
3. A aparência do corpo é o principal motivo de bullying?
4. O racismo é frequente em relação aos principais motivos de bullying?
5. Os alunos de escola pública costumam gastar mais tempo de deslocamento no dia a dia?
6. Uma pesquisa recente noticiada na revista [Galileu](https://revistagalileu.globo.com/Ciencia/Saude/noticia/2020/12/jovens-brasileiros-estao-mais-altos-mas-nao-necessariamente-mais-saudaveis.html) aponta que as meninas de 19 anos têm em média 23,30 de IMC. A amostra utilizada condiz com a pesquisa (mesmo contendo dados de 2015)?
7. A probabilidade de encontrar meninas de 19 anos com IMC próximo ao do estudo citado da revista Galileu é maior nas escolas privadas?

# **Análise dos Dados**

## Importação de Bibliotecas

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import datetime
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.stats.weightstats import ztest

## Importação de Dados

In [9]:
#base de dados da Amostra 2
url = 'https://raw.githubusercontent.com/andrevcmelo/BootcampAlura2021/main/bases_modulo3/PENSE_AMOSTRA2_ALUNOESCOLA.CSV'

df = pd.read_csv(url, sep=';',thousands=".", decimal=",")

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16556 entries, 0 to 16555
Columns: 296 entries, ANOPESQ to turma
dtypes: float64(10), int64(286)
memory usage: 37.4 MB


In [11]:
df.head(5)

,ANOPESQ,PAIS,REGEOGR,VB00004,VB01001,VB01002,VB01003,VB01004,VB01005,VB01006,VB01007,VB01008A,VB01010A,VB01011,VB01012,VB01013,VB01014,VB01015A,VB01016,VB01017,VB01018,VB01019,VB01020A,VB01021,VB01022,VB01023,VB01024,VB01025,VB01026,VB02001,VB02002,VB02004A,VB02010,VB02011,VB02013,VB02017A,VB02018A,VB02019A,VB02020A,VB02021,...,VE01P14A04,VE01P14A05,VE01P14A06,VE01P14A07,VE01P14A08,VE01P14A09,VE01P14A10,VE01P14A11,VE01P49,VE01P50,VE01P51,VE01P52,VE01P53,VE01P54,VE01P55,VE01P56,VE01P57,VE01P58,VE01P25,VE01P59,VE01P60,VE01P61,VE01P62,VE01P63,VE01P64,VE01P65,VE01P66,VE01P67,VE01P26,VE01P27,VE01P28A,VE01P68,VE01P69,VE01P70,VE01P71,VE01P72,V0041,aluno,escola,turma
0,2015,76,1,1,2,1,13,7,7,1,2,7,3,2,-1,1,1,1,1,1,2,4,2,4,1,2,2,5,1,8,3,4,3,6,6,1,6,1,-1,2,...,2,1,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,5,1,1,2,2,-1,2,2,2,1,1,2,2,1,1,1,1,1,1,1,1,1,296
1,2015,76,1,1,2,1,14,4,7,1,1,7,3,2,-1,1,1,1,1,1,2,2,2,4,1,2,2,5,3,8,2,1,8,8,6,1,1,5,-1,2,...,2,1,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,5,1,1,2,2,-1,2,2,2,1,1,2,2,1,1,1,1,1,1,1,2,1,296
2,2015,76,1,1,1,4,13,6,7,1,1,5,5,2,-1,2,1,1,1,1,2,3,1,4,1,2,2,5,3,2,3,1,8,2,8,1,1,5,5,1,...,2,1,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,5,1,1,2,2,-1,2,2,2,1,1,2,2,1,1,1,1,1,1,1,3,1,296
3,2015,76,1,1,1,1,14,6,7,1,1,7,4,2,-1,1,1,1,1,1,2,4,2,4,1,2,2,6,3,2,4,1,3,2,7,1,5,1,-1,2,...,2,1,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,5,1,1,2,2,-1,2,2,2,1,1,2,2,1,1,1,1,1,1,1,4,1,296
4,2015,76,1,1,1,1,13,9,7,1,2,5,4,2,-1,2,1,1,1,1,2,5,1,4,1,2,2,6,5,6,3,5,2,6,2,1,5,1,5,1,...,2,1,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,5,1,1,2,2,-1,2,2,2,1,1,2,2,1,1,1,1,1,1,1,5,1,296


## Tratamento dos dados

Como podemos ver, as colunas da base de dados estão em códigos, para facilitar a visualização e manipulação dos dados. Diante disto, usaremos o dicionário de dados fornecido pelo IBGE para renomear as colunas e alguns categóricos que iremos utilizar nas análises.

In [12]:
#Dicionários para auxiliar nas análises
regiao = {
    1: 'Norte',
    2: 'Nordeste',
    3: 'Sudeste',
    4: 'Sul',
    5: 'Centro-Oeste'
}
sexo = {
    1: 'Masculino',
    2: 'Feminino'
}
raca = {
    1: 'Branca',
    2: 'Preta',
    3: 'Amarela',
    4: 'Parda',
    5: 'Indígena',
    99: 'Não informado'
}
turno = {
    1: 'Manhã',
    2: 'Intermediário',
    3: 'Tarde',
    4: 'Noite',
    5: 'Integral',
    99: 'Não informado'
}
serie = {
    1: '6º ano Ensino Fundamental',
    2: '7º ano Ensino Fundamental',
    3: '8º ano Ensino Fundamental',
    4: '9º ano Ensino Fundamental',
    5: '1º ano Ensino Médio',
    6: '2º ano Ensino Médio',
    7: '3º ano Ensino Médio',
    99: 'Não informado'
}
situacao_escola = {
    1: 'Urbana',
    2: 'Rural'
}
tipo_escola = {
    1: 'Pública',
    2: 'Privada'
}
satisfacao = {
    1: 'Muito satisfeito(a)',
    2: 'Satisfeito(a)',
    3: 'Indiferente',
    4: 'Insatisfeito(a)',
    5: 'Muito insatisfeito(a)',
    99: 'Não informado'
}
frequencia = {
    1: 'Nunca',
    2: 'Raramente',
    3: 'Às vezes',
    4: 'Na maior parte do tempo',
    5: 'Sempre',
    99: 'Não informado'
}
motivo_bullying = {
    -1: 'Pulo no questionário',
    1: 'A minha cor ou raça',
    2: 'A minha religião',
    3: 'A aparência do meu rosto',
    4: 'A aparência do meu corpo',
    5: 'A minha orientação sexual',
    6: 'A minha região de origem',
    7: 'Outros motivos/causas',
    99: 'Não informado'
}
sofreu_bullying = {
    1: 'Sim',
    2: 'Não',
    3: 'Não sei o que é bullying',
    99: 'Não informado'
}

In [13]:
#Criando colunas "traduzidas" de acordo com o Dicionário de dados e inserindo em um novo DataFrame para facilitar as análises
dados = pd.DataFrame()
dados['Regiao'] = df['REGEOGR'].map(regiao)
dados['Sexo'] = df['VB01001'].map(sexo)
dados['Raça'] = df['VB01002'].map(raca)
dados['Idade'] = df['VB01003']
dados['Serie'] = df['VB01021'].map(serie)
dados['Turno'] = df['VB01022'].map(turno)
dados['DeslocDiario'] = df['TEMPODESLOC']
dados['SituacaoEscola'] = df['V0006'].map(situacao_escola)
dados['TipoEscola'] = df['V0007'].map(tipo_escola)
dados['Altura'] = round(df['VB17004']/100,2)
dados['Peso'] = round(df['VB17003'],2)
dados['Satisfacao_Corpo'] = df['VB11007'].map(satisfacao)
dados['Ja_Sofreu_Bullying'] = df['VB07010'].map(sofreu_bullying)
dados['Frequencia_Bullying'] = df['VB07007'].map(frequencia)
dados['Motivo_Bullying'] = df['VB07008'].map(motivo_bullying)

dados.head(5)

,Regiao,Sexo,Raça,Idade,Serie,Turno,DeslocDiario,SituacaoEscola,TipoEscola,Altura,Peso,Satisfacao_Corpo,Ja_Sofreu_Bullying,Frequencia_Bullying,Motivo_Bullying
0,Norte,Feminino,Branca,13,9º ano Ensino Fundamental,Manhã,0.0,Urbana,Privada,1.57,37.1,Satisfeito(a),Não,Nunca,Pulo no questionário
1,Norte,Feminino,Branca,14,9º ano Ensino Fundamental,Manhã,120.0,Urbana,Privada,1.52,50.2,Satisfeito(a),Não,Nunca,Pulo no questionário
2,Norte,Masculino,Parda,13,9º ano Ensino Fundamental,Manhã,0.0,Urbana,Privada,1.61,48.2,Satisfeito(a),Sim,Raramente,Outros motivos/causas
3,Norte,Masculino,Branca,14,9º ano Ensino Fundamental,Manhã,0.0,Urbana,Privada,1.72,69.4,Satisfeito(a),Sim,Nunca,Pulo no questionário
4,Norte,Masculino,Branca,13,9º ano Ensino Fundamental,Manhã,30.0,Urbana,Privada,1.68,78.9,Insatisfeito(a),Sim,Raramente,A aparência do meu corpo


Criando uma features para maximizar as possibilidades de análise, que são:
1. IMC (Índice de Massa Corporal)
2. Definição de Classes de IMC
3. Classes de Altura
4. Classes de Peso

**Extra**: [Regra de Sturges](https://maestrovirtuale.com/regra-de-sturges-explicacao-aplicacoes-e-exemplos/#:~:text=A%20regra%20Sturges%20%C3%A9%20um%20m%C3%A9todo%20emp%C3%ADrico%20amplamente%20usado%20em,representando%20uma%20amostra%20ou%20popula%C3%A7%C3%A3o.) é um critério usado para determinar o número de classes ou intervalos necessários para representar graficamente um conjunto de dados estatísticos.

In [14]:
# 1.
dados['IMC'] = round(dados['Peso'] / dados['Altura']**2,2)

#2.
classes_IMC = [0, 18, 25, 30, 60]
rotulos_IMC = ['Desnutrido', 'Normal', 'Sobrepeso', 'Obesidade']
dados['Classe_IMC'] = pd.cut(x = dados['IMC'], bins = classes_IMC, labels = rotulos_IMC, include_lowest=True)

#K = número de intervalos de categoria para auxiliar em análises, utilizando a Regra de Sturges
k = int(1 + ((10/3) * np.log10(len(dados))))

#3.
dados['Classe_Altura'] = pd.cut(x = dados['Altura'], bins = k, include_lowest=True)

#4.
dados['Classe_Peso'] = pd.cut(x = dados['Peso'], bins = k, include_lowest=True)

dados.head(5)

,Regiao,Sexo,Raça,Idade,Serie,Turno,DeslocDiario,SituacaoEscola,TipoEscola,Altura,Peso,Satisfacao_Corpo,Ja_Sofreu_Bullying,Frequencia_Bullying,Motivo_Bullying,IMC,Classe_IMC,Classe_Altura,Classe_Peso
0,Norte,Feminino,Branca,13,9º ano Ensino Fundamental,Manhã,0.0,Urbana,Privada,1.57,37.1,Satisfeito(a),Não,Nunca,Pulo no questionário,15.05,Desnutrido,"(1.524, 1.571]","(34.94, 46.32]"
1,Norte,Feminino,Branca,14,9º ano Ensino Fundamental,Manhã,120.0,Urbana,Privada,1.52,50.2,Satisfeito(a),Não,Nunca,Pulo no questionário,21.73,Normal,"(1.477, 1.524]","(46.32, 57.7]"
2,Norte,Masculino,Parda,13,9º ano Ensino Fundamental,Manhã,0.0,Urbana,Privada,1.61,48.2,Satisfeito(a),Sim,Raramente,Outros motivos/causas,18.59,Normal,"(1.571, 1.619]","(46.32, 57.7]"
3,Norte,Masculino,Branca,14,9º ano Ensino Fundamental,Manhã,0.0,Urbana,Privada,1.72,69.4,Satisfeito(a),Sim,Nunca,Pulo no questionário,23.46,Normal,"(1.713, 1.761]","(69.08, 80.46]"
4,Norte,Masculino,Branca,13,9º ano Ensino Fundamental,Manhã,30.0,Urbana,Privada,1.68,78.9,Insatisfeito(a),Sim,Raramente,A aparência do meu corpo,27.95,Sobrepeso,"(1.666, 1.713]","(69.08, 80.46]"


## Análises

###### **1) Os adolescentes brasileiros estão satisfeitos com seus corpos?**

###### **2) Há uma alta frequência de bullying nas escolas?**

###### **3) A aparência do corpo é o principal motivo de bullying?**

###### **4) O racismo é frequente em relação aos principais motivos de bullying?**

###### **5) Os alunos de escola pública costumam gastar mais tempo de deslocamento no dia a dia?**

###### **6) Uma pesquisa recente noticiada na revista Galileu aponta que as meninas de 19 anos têm em média 23,30 de IMC. A amostra utilizada condiz com a pesquisa (mesmo contendo dados de 2015)?**

###### **7) A probabilidade de encontrar meninas de 19 anos com IMC próximo ao do estudo citado da revista Galileu é maior nas escolas privadas?**